In [1]:
import numpy
import pandas as pd
import statsmodels.formula.api as smf # for anova
import statsmodels.stats.multicomp as multi 

In [10]:
# read the data
data = pd.read_csv("addhealth.csv", low_memory=False)

The National Longitudinal Study of Adolescent Health (AddHealth) is a representative school-based survey of adolescents in grades 7-12 in the United States. The Wave 1 survey focuses on factors that may influence adolescents’ health and risk behaviors, including personal traits, families, friendships, romantic relationships, peer groups, schools, neighborhoods, and communities. Check out the AddHealth Code Book Index to see a list of the topics addressed in each codebook section.

**Response variable**: Number of alcohols drunk in a day (H1TO16) - range 1-90

**Explanatory variables**:
1. Whether or not a student has been expelled (H1ED9) - value {0,1}
2. Religion (H1RE1) - value in range 1-27, we skip "other religion" (#28)


In [27]:
# Question1: whether or not a student has been expelled correlated 
#            with alcohol quantity among students in grades 7-12

# subset data frame with our variable of interest
sub1 = data[["H1TO16", "H1ED9"]] \
    .rename(columns={"H1TO16": "num_alcohol", "H1ED9": "expelled"}) \
    .apply(pd.to_numeric, errors='coerce') \
    .dropna() \
    .astype(int)

sub1 = sub1[(sub1["num_alcohol"] >= 1) & (sub1["num_alcohol"] <= 90) & \
            ((sub1["expelled"] == 0) | (sub1["expelled"] == 1))]

sub1

,num_alcohol,expelled
0,5,0
1,10,0
4,1,0
7,3,0
18,12,0
...,...,...
6482,1,0
6484,4,0
6492,2,0
6501,5,0


In [28]:
model1 = smf.ols(formula='num_alcohol ~ C(expelled)', data=sub1)
results1 = model1.fit()
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            num_alcohol   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     10.35
Date:                Mon, 06 Feb 2023   Prob (F-statistic):            0.00131
Time:                        13:54:30   Log-Likelihood:                -9841.1
No. Observations:                2909   AIC:                         1.969e+04
Df Residuals:                    2907   BIC:                         1.970e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.3226      0.136     39.095      0.000       5.056       5.590
C(expelled)[T.1]     1.8340      0.570      3.218      0.001       0.716       2.951
==============================================================================
Omnibus:                     3101.144   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           238795.153
Skew:                           5.301   Prob(JB):                         0.00
Kurtosis:                      46.101   Cond. No.                         4.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Conclusion:
whether or not a student has been expelled is correlated with alcohol quantity among students in grades 7-1, i.e., we reject null hypothesis

In [41]:
# Question2: do religions correlated with alcohol quantity among students in grades 7-12?

# subset data frame with our variable of interest
sub2 = data[["H1TO16", "H1RE1"]] \
    .rename(columns={"H1TO16": "num_alcohol", "H1RE1": "religion"}) \
    .apply(pd.to_numeric, errors='coerce') \
    .dropna() \
    .astype(int)

sub2 = sub2[(sub2["num_alcohol"] >= 1) & (sub2["num_alcohol"] <= 90) & \
            (sub2["religion"] >= 1) & (sub2["religion"] <= 27)]

sub2

,num_alcohol,religion
4,1,5
7,3,22
19,2,13
20,4,12
21,1,22
...,...,...
6477,3,4
6478,2,4
6482,1,11
6484,4,19


In [46]:
model2 = smf.ols(formula='num_alcohol ~ C(religion)', data=sub2).fit()
print (model2.summary())

print ('means for num_alcohol by major religion')
m2= sub2.groupby('religion').mean()
print (m2)

print ('standard deviations for num_alcohol by major religion')
sd2 = sub2.groupby('religion').std()
print (sd2)

                            OLS Regression Results                            
Dep. Variable:            num_alcohol   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.367
Date:                Mon, 06 Feb 2023   Prob (F-statistic):              0.106
Time:                        14:12:00   Log-Likelihood:                -8131.7
No. Observations:                2393   AIC:                         1.632e+04
Df Residuals:                    2367   BIC:                         1.647e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             4.0000      2.19

In [87]:
mc1 = multi.MultiComparison(sub2['num_alcohol'], sub2['religion'])
res1 = mc1.tukeyhsd()
res1df = pd.read_html(res1.summary().as_html(), header=0, index_col=0)[0].reset_index()
# Show 10 combinations with the smallest p-values
res1df.sort_values("p-adj").nsmallest(10,"p-adj")

,group1,group2,meandiff,p-adj,lower,upper,reject
193,10,15,27.5625,0.0513,-0.0548,55.1798,False
208,11,15,26.4815,0.0718,-0.8029,53.7659,False
264,15,21,-26.8182,0.0827,-54.8023,1.1659,False
222,12,15,26.0833,0.0869,-1.2619,53.4286,False
160,8,15,25.7234,0.0894,-1.3529,52.7997,False
235,13,15,25.4912,0.0924,-1.4188,52.4012,False
260,15,17,-25.4143,0.0984,-52.3977,1.5692,False
60,3,15,25.5417,0.1075,-1.8036,52.8869,False
268,15,26,-25.4333,0.1078,-52.6690,1.8023,False
266,15,23,-26.0000,0.1137,-53.9841,1.9841,False


### Conclusion:
religions are not correlated with alcohol quantity among students in grades 7-1, i.e., we DON'T reject null hypothesis